In [1]:
import json, tomllib
from proto import Message

with open("../config.toml", "rb") as f:
    config = tomllib.load(f)

In [67]:
project_id = config["global"]["project_id"]
location = config["global"]["location"]

# Vertex Search configurations
datastore_location = config["global"]["datastore_location"]
serving_config_id = config["global"]["serving_config_id"]

from google.cloud import discoveryengine_v1beta as discoveryengine
from google.cloud.discoveryengine_v1beta.services.search_service.pagers import (
    SearchPager,
)

search_client = discoveryengine.SearchServiceClient()


def vertexai_search_oneturn(
    search_query: str,
    include_content_spec: bool = True,
    include_citations: bool = True,
    summary_result_count: int = 3,
    return_snippet: bool = True,
    page_size: int = 10,
    search_filter: str = "",
    datastore_id: str = ""
) -> SearchPager:
    """
    Args:
        search_query: str
            The search query.
        return_snippet: bool
            Whether to return the snippet of the document. Default is True.
        summary_result_count: int
            The number of summary results to return. Default is 2.
        include_citations: bool
            Whether to include citations. Default is True.

    Returns:
        search_service.pagers.SearchPager
            Response message for [SearchService.Search] method.

            Iterating over this object will yield results and resolve
            additional pages automatically.
    """
    serving_config = search_client.serving_config_path(
        project=project_id,
        location=datastore_location,
        data_store=datastore_id,
        serving_config=serving_config_id,
    )

    content_spec = None
    if include_content_spec:
        snippet_spec = (
            discoveryengine.SearchRequest.ContentSearchSpec.SnippetSpec(
                return_snippet=return_snippet
            )
        )
        summary_spec = (
            discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec(
                summary_result_count=summary_result_count,
                include_citations=include_citations,
            )
        )
        content_spec = discoveryengine.SearchRequest.ContentSearchSpec(
            snippet_spec=snippet_spec, summary_spec=summary_spec
        )

    request = discoveryengine.SearchRequest(
        content_search_spec=content_spec,
        serving_config=serving_config,
        query=search_query,
        page_size=page_size,
        filter=search_filter,
    )

    response = search_client.search(request)

    return response

In [89]:
search_response = vertexai_search_oneturn(
    search_query="How do I return a product?",
    summary_result_count=5,
    datastore_id=config["search-persona5"]["conversations_datastore_id"],
    page_size=20
)

In [97]:
responses = {}
responses["summary"] = search_response.summary.summary_text
responses["user_input"] = "How do I return a product?"

responses["search_results"] = []
for result in search_response.results:
    search_result_dict: dict = Message.to_dict(result) # type: ignore
    document = search_result_dict.get("document", {})
    derived_struct_data = document.get("derived_struct_data", {})
    
    if len(derived_struct_data.get("snippets", [])) > 0:
        struct_data = document.get("struct_data",{})
        responses["search_results"].append({
                "snippet": derived_struct_data["snippets"][0]["snippet"],
                "link": derived_struct_data["link"],
                "conversation_id": search_result_dict["id"],
                "title": struct_data["title"],
                "status": struct_data["status"],
                "sentiment": struct_data["sentiment"],
                "rating": struct_data["rating"],
                "product_id": struct_data["product_id"],
                "customer_id": struct_data["customer_id"],
                "customer_email": struct_data["customer_email"],
                "conversation": struct_data["conversation"],
                "category": struct_data["category"],
                "agent_id": struct_data["agent_id"],
                "agent_email": struct_data["agent_email"]
            }
        )

In [105]:
responses

{'summary': 'To return a product, you can either bring it to a physical store or send it back to us via mail [1]. If you choose to mail it back, you&#39;ll need to securely pack the product in its original packaging and include a copy of your receipt or order confirmation [1]. You&#39;ll also need to provide your full name, contact information, and the reason for the return [2].',
 'user_input': 'How do I return a product?',
 'search_results': [{'snippet': 'According to our policy, you can return the product within 30 days of the ... Once we receive the <b>returned product</b> and verify its condition, we&#39;ll process&nbsp;...',
   'link': 'gs://csm-dataset/p5-dataset/html_conversations/11437.html',
   'conversation_id': '11437',
   'title': 'Black Gas Grill Perfect for Entertaining',
   'status': 'not resolved',
   'sentiment': 'positive',
   'rating': '4',
   'product_id': '519',
   'customer_id': '181',
   'customer_email': 'user1168@sample_user1168.sample',
   'conversation': " *

# Validate Conversation

In [106]:
converse_client = discoveryengine.ConversationalSearchServiceClient()

datastore_id = config["search-persona5"]["conversations_datastore_id"]
converse_request = discoveryengine.CreateConversationRequest(
    parent=(
        f"projects/{project_id}/"
        f"locations/{datastore_location}/"
        "collections/default_collection/"
        f"dataStores/{datastore_id}"
    ),
    conversation=discoveryengine.Conversation(
        user_pseudo_id="user@google.com"
    ),
)
conversation = converse_client.create_conversation(
    request=converse_request
)

In [107]:
def vertexai_search_multiturn(
    search_query: str,
    conversation_id: str,
    include_citations: bool = True,
    summary_result_count: int = 5,
    datastore_id: str = "",
    user_email: str = "user@example.com",
    write_user_events: bool = True,
    # search_filter: str = ""   # Uncomment when available in Proto
) -> discoveryengine.ConverseConversationResponse:
    """Searches for documents on Vertex AI Search using a conversational
    interface.

    Args:
        search_query (str):
            User query
        conversation_id (str):
            Vertex AI Search conversation identifier
        include_citations (bool):
            Include or not citations
            Default: True
        filter (str):
            Filter expression to the search responses
            Default: ""

    Returns:
        (discoveryengine.ConverseConversationResponse)
        Response for the query.
    """
    serving_config = converse_client.serving_config_path(
        project=project_id,
        location=datastore_location,
        data_store=datastore_id,
        serving_config=serving_config_id,
    )

    summary_spec = discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec(
        include_citations=include_citations,
        summary_result_count=summary_result_count,
    )

    request = discoveryengine.ConverseConversationRequest(
        name=conversation_id,
        query=discoveryengine.TextInput(input=search_query),
        serving_config=serving_config,
        summary_spec=summary_spec,
        # search_filter=search_filter,  # Uncomment when available in Proto
    )

    response = converse_client.converse_conversation(request)
    return response


In [109]:
conv_result = vertexai_search_multiturn(
    search_query="How do I return a product?",
    conversation_id=conversation.name,
    datastore_id=datastore_id
)

In [112]:
conv_dict = Message.to_dict(conv_result)

# Search Manuals

In [3]:
import json, tomllib
from proto import Message
from google.cloud import discoveryengine_v1beta as discoveryengine
from google.cloud.discoveryengine_v1beta.services.search_service.pagers import (
    SearchPager,
)

search_client = discoveryengine.SearchServiceClient()

with open("../config.toml", "rb") as f:
    config = tomllib.load(f)

project_id = config["global"]["project_id"]
location = config["global"]["location"]
datastore_location = config["global"]["datastore_location"]
serving_config_id = config["global"]["serving_config_id"]

custom_category = ""
user_query = "How do I install a bed?"

def search_manuals():
    search_filter = ""
    if custom_category:
        search_filter += f'category: ANY("{custom_category}") '

    search_response = vertexai_search_oneturn(
        search_query=user_query,
        summary_result_count=5,
        search_filter=search_filter,
        datastore_id=config["search-persona5"]["product_manuals_datastore_id"]
    )

    responses = {}
    responses["summary"] = search_response.summary.summary_text
    responses["user_input"] = user_query

    responses["search_results"] = []
    for result in search_response.results:
        search_result_dict: dict = Message.to_dict(result) # type: ignore
        document = search_result_dict.get("document", {})
        derived_struct_data = document.get("derived_struct_data", {})
        
        if len(derived_struct_data.get("snippets", [])) > 0:
            struct_data = document.get("struct_data",{})
            responses["search_results"].append({
                    "id": search_result_dict["id"],
                    "snippet": derived_struct_data["snippets"][0]["snippet"],
                    "link": derived_struct_data["link"],
                    "title": struct_data["title"],
                    "category": struct_data["category"],
                    "manual": struct_data["manual"]
                }
            )

    return responses


def vertexai_search_oneturn(
    search_query: str,
    include_content_spec: bool = True,
    include_citations: bool = True,
    summary_result_count: int = 3,
    return_snippet: bool = True,
    page_size: int = 10,
    search_filter: str = "",
    datastore_id: str = ""
) -> SearchPager:
    """
    Args:
        search_query: str
            The search query.
        return_snippet: bool
            Whether to return the snippet of the document. Default is True.
        summary_result_count: int
            The number of summary results to return. Default is 2.
        include_citations: bool
            Whether to include citations. Default is True.

    Returns:
        search_service.pagers.SearchPager
            Response message for [SearchService.Search] method.

            Iterating over this object will yield results and resolve
            additional pages automatically.
    """
    serving_config = search_client.serving_config_path(
        project=project_id,
        location=datastore_location,
        data_store=datastore_id,
        serving_config=serving_config_id,
    )

    content_spec = None
    if include_content_spec:
        snippet_spec = (
            discoveryengine.SearchRequest.ContentSearchSpec.SnippetSpec(
                return_snippet=return_snippet
            )
        )
        summary_spec = (
            discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec(
                summary_result_count=summary_result_count,
                include_citations=include_citations,
            )
        )
        content_spec = discoveryengine.SearchRequest.ContentSearchSpec(
            snippet_spec=snippet_spec, summary_spec=summary_spec
        )

    request = discoveryengine.SearchRequest(
        content_search_spec=content_spec,
        serving_config=serving_config,
        query=search_query,
        page_size=page_size,
        filter=search_filter,
    )

    response = search_client.search(request)

    return response

In [5]:
response = search_manuals()